# Assignment 1
Remote Sensing & Big Data
Assignment 1
In this assignment, you will implement the map-reduce principles and some basic image operations (e.g. morphological operations) on the Landsat 7 image collection in the Google Earth Engine (GEE). Below you find the different steps to perform and the questions that come with these steps.

When handing in the assignment on Brightspace, we ask you to upload the following:

- Your GEE scripts:
- When working in the javascript/web-interface: please add your code as appendix to your report (e.g. printed as pdf) and a hyperlink to the working code on GEE.
- When working in the python-interface: please add your jupyter-notebook and a printed version (e.g. as pdf) of this notebook.
- A motivation for each step in your code. This can be done as comments in your code or separate in the report.
- Response to the questions, which are highlighted in red.

If you do not provide your code and/or motivation for each step, we will not grade the assignment resulting in a fail score.


In [26]:
import ee
import geemap

In [27]:
#ee.Authenticate()
#ee.Initialize()

## Step 1 Data loading
- Load the USGS Landsat 7 Top of Atmosphere Reflectance Tier 1 data over a region of your interest. FYI: if you want to know what the difference between the different Tiers is: https://www.usgs.gov/media/videos/landsat-collections-what-are-tiers

In [28]:
house_coords = [37.9548, -79.4556]  # lat long
box_size= 0.5  # in degrees
bbox_coord = [house_coords[1]-box_size,house_coords[0]-box_size,house_coords[1]+box_size,house_coords[0]+box_size]

roi = ee.Geometry.BBox(*bbox_coord)
ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA").filterBounds(roi)
Map = geemap.Map(center = house_coords,zoom=10)
Map.addLayerControl()

## Step 2 Naive annual composites
- Apply the map-reduce principle to convert the image collection of Step 1 into annual composites with different filtering & reducing conditions. Within this step, implement:
    - A: Different filtering conditions based cloud cover metadata:
        - a1: no filtering

In [29]:
a1 = ls.max()

v = {'bands':['B3','B2','B1'],
        #  'min':0,
         # 'max':0.4,
        'gamma': 1.2,
         #'bias':-0.2,
         #'gain': 0.1
    }
Map.add_ee_layer(
    ee_object=a1,
    vis_params=v,
    name='a1',
)

Map

Map(center=[37.9548, -79.4556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

        - a2: filtering on <50% cloud cover 

In [30]:
a2 = ls.filterMetadata('CLOUD_COVER','less_than',50)

Map.add_ee_layer(
    ee_object=a2,
    vis_params=v,
    name='a2',
)

Map

Map(center=[37.9548, -79.4556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

        - a3: filtering in <20% cloud cover
        

In [31]:
a3 = ls.filterMetadata('CLOUD_COVER','less_than',20).median()

v = {'bands':['B3','B2','B1'],
          'min':0,
          'max':0.4,
        'gamma': 1.2
    }
Map.add_ee_layer(
    ee_object=a3,
    vis_params=v,
    name='a3',
)

Map

Map(center=[37.9548, -79.4556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

- B: Use the result of a2 and test the effect of masking the clouds & cloud-shadows using the BQA bitmask: 
    - b0: without cloud masking 

In [32]:
Map = geemap.Map(center = house_coords,zoom=10)
Map.addLayerControl()

In [33]:
b0 = a2

v = {'bands':['B3','B2','B1'],
          'min':0,
          'max':0.4,
        'gamma': 1.2
    }
Map.add_ee_layer(
    ee_object=b0,
    vis_params=v,
    name='b0',
)

Map

Map(center=[37.9548, -79.4556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

    - b1: with the clouds masked 

In [38]:
print(1 << 3)


8


In [41]:
def cld_mask(image: ee.Image):
    cloudbit = 1 << 3
    bqa = image.select('BQA')
    cloudmask = bqa.bitwise_and(cloudbit).eq(0)
    cloudmask = cloudmask.rename(['cloudMask'])
    image = image.addBands(cloudmask)
    image.updateMask('cloudMask')
    return image
    
def cld_shdw_mask(image):
    cloudbit = 1 << 3
    cloudshadowbit = 3 << 6
    bqa = image.select('BQA')
    cloudmask = bqa.bitwise_and(cloudbit+cloudshadowbit).eq(0)
    image.addBands(image.updateMask(cloudmask),names=['CloudShadowMask'])
    return image

In [57]:
# testing the cloud mask layer
masktest = cld_mask(a1)
Map = geemap.Map(center = house_coords,zoom=10)
v = {'bands':['B3','B2','B1'],
          'min':0,
          'max':0.5,
        'gamma': 1.2
    }
Map.add_ee_layer(masktest,vis_params=v)
Map.add_ee_layer(masktest)
Map

Map(center=[37.9548, -79.4556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [ ]:
b1 = b0.map(cld_mask)



In [ ]:
v = {'bands':['B3','B2','B1'],
          'min':0,
          'max':0.5,
        'gamma': 1.2
    }
Map.add_ee_layer(
    ee_object=b1,
    #vis_params=v,
    name='b1',
)

Map

    - b2: with clouds and shadow masked
    

In [ ]:
b2 = b0.map(cld_shdw_mask)

v = {'bands':['B3','B2','B1'],
          'min':0,
          'max':0.5,
        'gamma': 1.2
    }
Map.add_ee_layer(
    ee_object=b2,
    vis_params=v,
    name='b2',
)
Map.add_ee_layer(b2,{'bands':['cloudMask']})
Map

- C: For every step of A&B visualize different reducers for the RGB-bands: 
    - mean 
    - median 
    - count
        - 0.05 percentile
        - 0.95 percentile
- Visualize the results as a true color (for the mean, median and percentiles) and pseudo-color image for a pre- and post-SLC failure year (More background on the SLC-failure in https://www.usgs.gov/core-science-systems/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con )
- **Q1: What are the differences you observe between the different composites (e.g. number of scenes, cloud/noise/SLC artefacts, reflectance values and visual appearance) and explain their cause?**
- **Q2: Discuss what the impact of these differences could be on subsequent big data analysis and how it might affect the veracity of the analysis.**

## Step 3 Greenest pixel composites
- Use the map-principle to calculate the normalized difference vegetation index (NDVI) for every image and use this index to reduce the collection to a greenest pixel composite for every year.
- Visualize the results and compare them to the results of Step 2.
- **Q3: What are the differences between the greenest pixel composite and the results of step 2 and explain their cause?**
- **Q4: What are the advantages and potential disadvantages of using a quality mosaic?**

## Step 4 SLC- and cloud masking
- Load the cloud+cloud-shadow mask for every image by extracting the BQA mask.
- Apply different image dilation/ erosion / opening / closing operations on the BQA mask with different distances/iterations.
- Update the mask with the dilated/ erosed / opened / closed BQA mask.
- Pick one image and visualize the effect of the dilation/ erosion / opening / closing on the data.
- **Q5: Visualize and explain the differences of the different dilation/erosion/ opening/closing operations on the mask. What would be the different dis/advantages of the different methods?**

## Step 5 SLC-filling
- Apply a map-reduce function to the USGS Landsat 7 Top of Atmosphere Reflectance Tier 1 since 2003 where you replace the missing SLC/cloud/shadow data of every image by:
    - The median reflectance
    - The mean reflectance
    - The reflectance of the greenest pixel
    - The reflectance of the pixel that is closest in time
- Visualize two images (summer vs winter) where you illustrate the effect of the different filling scenarios.
- **Q6: Visualize and explain the differences of the different filling scenarios. What would be the different dis/advantages of the different methods?**